# Greenhouse gas emissions to concentrations

Sometimes we don't want to run a 43-gas scenario and just want a subset.

Here I will run 5 gases: CO2, CH4, N2O, CFC11 and HFC134a.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d
import pickle
from climateforcing.utils import mkdir_p

from fair21.constants.gases import molwt, burden_per_emission, lifetime
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha
)
from fair21.gas_cycle.forward import step_concentration

In [ ]:
gas_list = ['CO2', 'CH4', 'N2O', 'CFC-11', 'HFC-134a']

In [ ]:
# grab some emissions
emissions = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    emissions[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()
    
# CO2 and N2O units need to behave
emissions["CO2"] = emissions["CO2"] / 1000
emissions["N2O"] = emissions["N2O"] / 1000
#co2_emissions = 1/1000 * df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|CO2') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()
#ch4_emissions = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|CH4') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()
#cfc11_emissions = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|Montreal Gases|CFC|CFC11') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
concentration = {}
airborne_emissions = {}
cumulative_emissions = {}

In [ ]:
# grab some concentrations
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    concentration_rcmip[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

In [ ]:
n_gases = len(gas_list)
n_gases

In [ ]:
n_timesteps = len(emissions['CFC-11'])

In [ ]:
%%time

# initialise arrays. Using arrays makes things run quicker
emissions_array = np.ones((n_timesteps, n_gases)) * np.nan
concentration_array = np.ones((n_timesteps, n_gases)) * np.nan
g0 = np.ones((n_gases)) * np.nan
g1 = np.ones((n_gases)) * np.nan
alpha_lifetime = np.ones((n_gases))
airborne_emissions = np.zeros((n_gases))
gas_boxes = np.zeros((n_gases, 4))  # TODO: 4 needs to be unhardcoded
cumulative_emissions_array = np.ones((n_timesteps, n_gases)) * np.nan
iirf_0_array = np.ones(n_gases) * np.nan
iirf_cumulative_array = np.ones(n_gases) * np.nan
iirf_temperature_array = np.ones(n_gases) * np.nan
iirf_airborne_array = np.ones(n_gases) * np.nan
burden_per_emission_array = np.ones(n_gases) * np.nan
lifetime_array = np.ones((n_gases, 4)) * np.nan
pre_industrial_concentration_array = np.ones(n_gases) * np.nan
partition_fraction_array = np.zeros((n_gases, 4))
natural_emissions_adjustment_array = np.ones(n_gases) * np.nan

for i, gas in enumerate(gas_list):
    emissions_array[:, i] = emissions[gas]
    cumulative_emissions_array[:, i] = np.cumsum(emissions[gas])
    g0[i], g1[i] = calculate_g(lifetime[gas], partition_fraction=partition_fraction[gas])
    iirf_0_array[i] = iirf_0[gas]
    iirf_cumulative_array[i] = iirf_cumulative[gas]
    iirf_temperature_array[i] = iirf_temperature[gas]
    iirf_airborne_array[i] = iirf_airborne[gas]
    burden_per_emission_array[i] = burden_per_emission[gas]
    lifetime_array[i, :] = lifetime[gas]
    pre_industrial_concentration_array[i] = pre_industrial_concentration[gas]
    partition_fraction_array[i, :] = partition_fraction[gas]
    natural_emissions_adjustment_array[i] = natural_emissions_adjustment[gas]

for i in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions_array[i],
        airborne_emissions,
        ssp245_temperature_rfmip[i],
        iirf_0_array,
        iirf_cumulative_array,
        iirf_temperature_array,
        iirf_airborne_array,
        g0,
        g1,
    )
    alpha_lifetime[np.isnan(alpha_lifetime)]=1  # CF4 seems to have an issue. Should we raise warning?
    concentration_array[i, :], gas_boxes, airborne_emissions = step_concentration(
        emissions_array[i, :, None], 
        gas_boxes,
        airborne_emissions, 
        burden_per_emission_array,
        lifetime_array,
        alpha_lifetime=alpha_lifetime[:, None],
        pre_industrial_concentration=pre_industrial_concentration_array,
        timestep=1,
        partition_fraction=partition_fraction_array,
        natural_emissions_adjustment=natural_emissions_adjustment_array[:, None],
    )

In [ ]:
# put back together
for i, gas in enumerate(gas_list):
    concentration[gas] = concentration_array[:, i]

In [ ]:
fig, ax = pl.subplots(2, 3, figsize=(16,9))
for igas, gas in enumerate(gas_list):
    iy = igas % 3
    ix = igas // 3
    ax[ix, iy].plot(np.arange(1750.5, 2501), concentration[gas], label='FaIR 2.1')
    ax[ix, iy].plot(np.arange(1750.5, 2501), concentration_rcmip[gas], label='History + MAGICC6')
    ax[ix, iy].set_title(gas)
fig.tight_layout()